# BI-Projekt
## Aufgabe 3

***3. Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht
abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?***

In [27]:
#Relevante module
import pandas as pd
import folium
from folium import plugins

Die atp_tourneys_till_2022 tabelle wurde angereichert mit Koordinaten. Problematisch sind aber hierbei Davis-Cup Spiele. Aufgrund des hohen Aufwandes werden die Koordinaten für diese Leer gelassen und später auch entfernt.

In [28]:
df_tourney_coord = pd.read_csv('data/atp_tourneys_till_2022_coordinates.csv')
df_tourney_coord.sample(5)

,tourney_id,tourney_name,tourney_location,surface,draw_size,tourney_level,tourney_date,latitude,longitude
3438,2003-D083,Davis Cup WG PO: ECU vs ROU,Davis Cup WG PO: ECU vs ROU,Clay,4,D,20030919,NaN,NaN
1351,1973-D020,Davis Cup EUR SF: MEX vs COL,Davis Cup EUR SF: MEX vs COL,Clay,4,D,19730323,NaN,NaN
3020,1998-D083,Davis Cup G2 F: URU vs VEN,Davis Cup G2 F: URU vs VEN,Clay,4,D,19980925,NaN,NaN
402,1987-328,Basel,Basel,Hard,32,A,19871005,47.558108,7.587826
3134,2000-D070,Davis Cup WG R1: CZE vs GBR,Davis Cup WG R1: CZE vs GBR,Clay,4,D,20000204,NaN,NaN


In [29]:
df_matches = pd.read_csv("data/atp_matches_till_2022.csv")
df_matches.sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
13311,1971-D045,Davis Cup EUR R1: ROU vs NED,Clay,4,D,19710430,2,100119,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160536,2012-429,Stockholm,Hard,28,A,20121015,13,104542,1.0,NaN,...,36.0,26.0,12.0,10.0,2.0,5.0,7.0,4810.0,57.0,761.0
103501,1995-D063,Davis Cup G2 QF: LAT vs UKR,Clay,4,D,19950714,2,102742,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,598.0,23.0,730.0,14.0
81430,1989-520,Roland Garros,Clay,128,G,19890529,109,100942,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,184.0,NaN,126.0,NaN
113535,1998-329,Tokyo,Hard,56,A,19980413,53,102318,16.0,NaN,...,32.0,24.0,13.0,10.0,2.0,6.0,109.0,439.0,81.0,578.0


Ebenso ist die atp_players_till_2022 erweitert worden um leichter auf das Land schließen zu können.

In [30]:
df_players = pd.read_csv('data/atp_players_till_2022_ioc2country_v2.csv', delimiter=';')

Die Fragestellung bezieht sich explizit auf die Deutschen. Als IOC code gibt es aber eine handvoll Einträge die relevanten sind neben GER, da die Daten auch soweit zurückgehen bis zur DDR/BRD, also dem geteilten Deutschland.  
Relevante IOC-Codes:  
EUA FRG GDR GER.

Als relevanter IOC-Code hat sich auch DEU herausgestellt in der player-Tabelle. DEU ist eigentlich kein echter Code und wurde vermutlich einfach nur falsch eingetragen.

In [31]:
df_players.loc[df_players.ioc.eq('DEU')]

,player_id,name_first,name_last,hand,dob,ioc,ioc_nation,height,wikidata_id
58036,211386,Alessio,Vasquez Gehrke,U,NaN,DEU,Germany,NaN,NaN
58037,211387,Jonas Pelle,Hartenstein,U,NaN,DEU,Germany,NaN,NaN
58146,211496,Philipp,Pavlenko,U,NaN,DEU,Germany,NaN,NaN
58675,212025,Yannik,Kelm,U,NaN,DEU,Germany,NaN,NaN


Für die Fragestellung werden nun alle relevanten Spieler gefiltert.

In [32]:
df_ger_players = df_players.loc[df_players.ioc.eq('GDR') | df_players.ioc.eq('GER') | df_players.ioc.eq('FRG')| df_players.ioc.eq('EUA') | df_players.ioc.eq('DEU')]
df_ger_players

,player_id,name_first,name_last,hand,dob,ioc,ioc_nation,height,wikidata_id
32,100033,Wilhelm,Bungert,R,19390401.0,GER,Germany,188.0,Q77579
33,100034,Christian,Kuhnke,L,19390413.0,GER,Germany,NaN,Q461558
55,100056,Harald,Elschenbroich,R,19410619.0,GER,Germany,NaN,Q5653740
84,100085,Hans Joachim,Ploetz,R,19440226.0,GER,Germany,NaN,NaN
136,100137,Hans Jurgen,Pohmann,R,19470523.0,GER,Germany,178.0,Q95218
...,...,...,...,...,...,...,...,...,...
58470,211820,Hans Joachim,Richter,U,NaN,GDR,East Germany,NaN,NaN
58539,211889,Gunter,Reimann,U,NaN,GER,Germany,NaN,NaN
58571,211921,Horst,Stahlberg,U,NaN,GDR,East Germany,NaN,NaN
58581,211931,Gunther,Sanders,U,NaN,GER,Germany,NaN,NaN


Für die Auswertung wird benötigt, ob ein deutscher Spieler überhaupt an einem Match teilgenommen hat.  
Dieses neue Feature braucht aber ein paar zwischenschritte bevor wir dieses erstellen können.  
Als ersten Schritt fügen wir ein Feature hinzu, das sagt, ob ein deutscher das Match gewonnen hat.

In [33]:
df_matches['ger_win'] = df_matches.winner_id.isin(df_ger_players.player_id)
df_matches.sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win
116473,1999-407,Rotterdam,Carpet,32,A,19990215,25,103498,NaN,NaN,...,35.0,14.0,13.0,4.0,8.0,39.0,996.0,25.0,1333.0,False
141016,2006-500,Halle,Grass,32,A,20060612,18,103694,7.0,NaN,...,37.0,15.0,14.0,5.0,10.0,29.0,1120.0,90.0,445.0,False
117231,1999-379,Prague,Clay,32,A,19990426,19,101897,NaN,NaN,...,18.0,10.0,12.0,3.0,11.0,46.0,889.0,27.0,1189.0,False
49655,1980-714,San Jose,Hard,32,A,19800310,27,100243,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
43569,1979-3934,WCT Challenge Cup,Carpet,8,A,19791204,5,100437,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,False


Als zweiten Schitt fügen wir nun ein Feature hinzu, das sagt, ob ein deutscher das Match verloren hat.

In [34]:
df_matches['ger_lose'] = df_matches.loser_id.isin(df_ger_players.player_id)
df_matches.sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win,ger_lose
57476,1982-809,Strasbourg WCT,Carpet,32,A,19820315,24,100656,2.0,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,False,False
168972,2015-560,US Open,Hard,128,G,20150831,112,103819,2.0,NaN,...,15.0,14.0,3.0,8.0,2.0,9065.0,29.0,1230.0,False,True
79110,1989-314,Gstaad,Clay,32,A,19890710,14,100652,NaN,NaN,...,NaN,NaN,NaN,NaN,90.0,NaN,434.0,NaN,False,True
164600,2014-D040,Davis Cup G2 PO: BOL vs GUA,Clay,4,D,20140404,1,105499,NaN,NaN,...,NaN,NaN,NaN,NaN,429.0,91.0,789.0,23.0,False,False
145402,2007-D083,Davis Cup G2 PO: NOR vs ALG,Hard,4,D,20070720,4,104467,NaN,NaN,...,NaN,NaN,NaN,NaN,319.0,110.0,737.0,21.0,False,False


Mit diesen 2 neuen Features lässt sich nun einfach ein 3 Feature erstellen, das als Flag dient, ob überhaupt ein Deutscher teilgenommen hat am Match.

In [35]:
#Ein Deutscher hat teilgenommen, wenn der Sieger ein Deutscher war oder der Verlierer ein Deutscher war, oder auch beides
df_matches['ger_participated'] = df_matches['ger_win'] | df_matches['ger_lose']

In [36]:
df_matches.loc[df_matches.ger_participated.eq(True)].sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win,ger_lose,ger_participated
3606,1968-319,Kitzbuehel,Clay,32,A,19680820,283,100161,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True
57120,1982-615,Dusseldorf,Clay,16,A,19820503,30,100300,NaN,NaN,...,NaN,NaN,NaN,45.0,NaN,68.0,NaN,False,True,True
118475,1999-418,Washington,Hard,56,A,19990816,19,102205,NaN,NaN,...,13.0,4.0,7.0,98.0,502.0,86.0,546.0,False,True,True
137295,2005-424,San Jose,Hard,32,A,20050207,19,103163,3.0,NaN,...,8.0,7.0,12.0,17.0,1365.0,168.0,261.0,True,False,True
161924,2013-773,Bucharest,Clay,28,A,20130422,25,104198,NaN,NaN,...,14.0,4.0,9.0,87.0,576.0,29.0,1260.0,False,True,True


Da bereits wie oben erwähnt der Davis-Cup besonders Schwierigkeiten macht bei der location, werden diese nun jetzt entfernt.

In [37]:
df_matches = df_matches[~df_matches['tourney_name'].str.contains('Davis')]

In [38]:
df_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win,ger_lose,ger_participated
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187948,2022-0605,Tour Finals,Hard,8,F,20221114,290,126094,6.0,NaN,...,13.0,2.0,5.0,7.0,3530.0,3.0,5350.0,False,False,False
187949,2022-0605,Tour Finals,Hard,8,F,20221114,289,104925,7.0,NaN,...,11.0,1.0,2.0,8.0,3320.0,3.0,5350.0,False,False,False
187950,2022-0605,Tour Finals,Hard,8,F,20221114,288,126094,6.0,NaN,...,17.0,6.0,9.0,7.0,3530.0,5.0,4065.0,False,False,False
187951,2022-0605,Tour Finals,Hard,8,F,20221114,287,104925,7.0,NaN,...,16.0,4.0,6.0,8.0,3320.0,5.0,4065.0,False,False,False


Es wird nun Gruppiert nach Turniername. Von besonderen Interesse sind je Turnier:  
* Wie viele Matches es insgeamt gibt
* Bei wie vielen Matches spielte ein Deutscher
* Wie viele Matches hatten die Deutschen gewonnen.

In [39]:
agg_df = df_matches.groupby('tourney_name').agg(total_games_in_tourney=('ger_participated', 'count'),
                                                total_games_with_ger=('ger_participated', 'sum'),
                                                wins=('ger_win', 'sum')).reset_index()


In [40]:
agg_df.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins
417,San Diego WCT,8,0,0
299,Mexico City,279,14,6
184,Halle,887,388,223
140,Delray Beach WCT,46,1,0
90,Cagliari,27,6,4


Es wird dem DataFrame ein neues Feature hinzugefügt für die Siegchance.

In [41]:
agg_df['success_rate'] = agg_df['wins'] / agg_df['total_games_with_ger'] * 100
agg_df.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate
435,Seoul Olympics,63,5,3,60.000000
396,Richmond,36,0,0,NaN
81,Brussels,635,93,60,64.516129
82,Brussels WCT,31,0,0,NaN
359,Oslo,31,5,4,80.000000


Zur Prüfung werden die Zeilen sortiert nach Siegrate und angezeigt.

In [42]:
agg_df.sort_values(by='success_rate', ascending=False)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate
136,Dallas WCT,151,3,3,100.000000
207,Indian Wells,165,14,12,85.714286
247,Laver Cup,40,7,6,85.714286
106,Charleston,77,6,5,83.333333
7,Aix-en-Provence,47,6,5,83.333333
...,...,...,...,...,...
523,Wembley 3,16,0,0,NaN
524,Wembley NTL 1,3,0,0,NaN
525,Wembley NTL 2,6,0,0,NaN
528,World Invitational Classic,15,0,0,NaN


Für die Fragestellung haben Turniere, bei denen es nur wenig Spiele mit Deutschen gibt nicht wirklich relevant. Ein Turnier bei zum Beispiel 4-mal gespielt wurde und 4-mal gewonnen wurde hat mit seiner 100 % Siegwahrscheinlichkeit eher eine geringe Aussage.  
  
Da es viele Turniere gibt mit nur kleiner Teilnehmeranzahl, wird das 90 % Quantil genutzt des Features total_games_with_ger(Anzahl Matches mit Deutschen).

In [43]:
quantile_90 = agg_df['total_games_with_ger'].quantile(0.90)
quantile_90

91.5

Das 90 % Quantil wird als Filter benutzt und es wird sortiert nach der Siegrate.

In [44]:
filtered_df = agg_df.loc[agg_df['total_games_with_ger'] > quantile_90]
filtered_df.sort_values(by='success_rate', ascending = False)


,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate
81,Brussels,635,93,60,64.516129
306,Milan,744,96,60,62.500000
209,Indianapolis,2375,159,93,58.490566
391,Queen's Club,2521,175,101,57.714286
184,Halle,887,388,223,57.474227
451,St. Petersburg,912,151,84,55.629139
506,Vienna,1552,265,146,55.094340
319,Moscow,917,143,78,54.545455
481,Tokyo,1187,182,99,54.395604
294,Memphis,1686,159,86,54.088050


Für die Fragestellungen sind hier jetzt relevante Turniere bei denen die Deutschen gut oder eher schlecht abschneiden.  Die atp_tourneys Tabelle ist angereichert worden mit Koordinaten. Diese werden benötigt für eine Geografische Darstellung.

In [45]:
df_coords = pd.read_csv('data/atp_tourneys_till_2022_coordinates.csv')
df_coords.sample(5)

,tourney_id,tourney_name,tourney_location,surface,draw_size,tourney_level,tourney_date,latitude,longitude
2893,1997-D007,Davis Cup G1 QF: AUT vs CRO,Davis Cup G1 QF: AUT vs CRO,Clay,4,D,19970404,NaN,NaN
7182,1987-433,Scottsdale,Scottsdale,Hard,32,A,19871005,33.494219,-111.926018
7983,1970-430,Wembley,Wembley,Carpet,32,A,19701116,51.553155,-0.295756
1850,1982-D049,Davis Cup WG PO: MEX vs ROU,Davis Cup WG PO: MEX vs ROU,Clay,4,D,19821001,NaN,NaN
4407,2015-D019,Davis Cup G1 PO: BAR vs ECU,Davis Cup G1 PO: BAR vs ECU,Hard,4,D,20150918,NaN,NaN


In [46]:
#Mergen für die Koordinaten
df_finished = pd.merge(filtered_df, df_coords[['tourney_name','tourney_location','latitude','longitude']], on='tourney_name',how='left').drop_duplicates(subset='tourney_name').sort_values(by='success_rate', ascending = False)

Die Fragestellung spricht von "... besonders gut oder besonders schlecht
abschneiden." Dafür haben Grenzen gesetzt je nach Siegrate:
* Ab über 55% Siegrate gelten die deutschen als sehr stark.
* Zwischen 55% und 52% als stark.
* Zwischen 48% und 45% als schwach
* Ab unter 45 als sehr schwach

In [47]:
df_ger_strong = df_finished.loc[df_finished.success_rate.gt(52) & df_finished.success_rate.le(55)]
df_ger_strong.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
1626,Tokyo,1187,182,99,54.395604,Tokyo,35.764784,139.148599
1429,Rome Masters,1950,184,99,53.804348,Rome,41.893320,12.482932
1128,Moscow,917,143,78,54.545455,Moscow,55.750446,37.617494
1603,Stuttgart Outdoor,937,295,156,52.881356,Stuttgart,48.778449,9.180013
974,Memphis,1686,159,86,54.088050,Memphis,35.149022,-90.051629


In [48]:
df_ger_very_strong = df_finished.loc[df_finished.success_rate.gt(55)]
df_ger_very_strong.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
632,Halle,887,388,223,57.474227,Halle,48.862204,2.347668
307,Brussels,635,93,60,64.516129,Brussels,50.846557,4.351697
804,Indianapolis,2375,159,93,58.490566,Indianapolis,39.768333,-86.158350
1726,Vienna,1552,265,146,55.094340,Vienna,48.208354,16.372504
1072,Milan,744,96,60,62.500000,Milan,45.463945,9.188558


In [49]:
df_ger_weak = df_finished.loc[df_finished.success_rate.lt(48) & df_finished.success_rate.ge(45)]
df_ger_weak.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
1040,Miami Masters,3040,318,145,45.597484,Miami,25.774173,-80.193620
1537,Stockholm,1763,193,88,45.595855,Stockholm,59.325117,18.071093
772,Indian Wells Masters,2512,279,131,46.953405,Indian Wells,33.721797,-116.338303
954,Madrid Masters,1044,94,45,47.872340,Madrid,40.416705,-3.703583
1884,s Hertogenbosch,685,112,52,46.428571,s Hertogenbosch,51.688935,5.303104


In [50]:
df_ger_very_weak = df_finished.loc[df_finished.success_rate.lt(45)]
df_ger_very_weak.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
697,Hamburg Masters,1101,247,105,42.510121,Hamburg,53.550341,10.000654
141,Barcelona,3081,261,117,44.827586,Barcelona,41.382894,2.177432
1462,Rotterdam,1409,172,73,42.441860,Rotterdam,51.924442,4.477750
1374,Roland Garros,6937,707,314,44.413013,Roland Garros,48.847300,2.248355
1582,Stuttgart,683,234,105,44.871795,Stuttgart,48.778449,9.180013


Jetzt sind alle DataFrames vorhanden mit allen benötigten Features. Mithilfe vom Modul folium werden die Turniere nun auf eine Karte platziert.

In [51]:
map = folium.Map(location=[df_ger_strong['latitude'].mean(), df_ger_strong['longitude'].mean()], zoom_start=4)

for index, row in df_ger_very_strong.iterrows():
    icon = folium.Icon(color='green', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
    
for index, row in df_ger_strong.iterrows():
    icon = folium.Icon(color='lightgreen', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
    
for index, row in df_ger_weak.iterrows():
    icon = folium.Icon(color='orange', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
    
for index, row in df_ger_very_weak.iterrows():
    icon = folium.Icon(color='red', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
map

Die Karte zeigt alle Turniere bei denen die Deutschen (sehr) stark oder (sehr) schwach waren(in Bezug auf die Siegrate).  
Es gilt:
* Dunkelgrün: Deutsche sind hier sehr stark
* Grün: Deutsche sind hier stark
* Orange: Deutsche sind hier schwach
* Rot: Deutsche sind hier sehr schwach

Folium erlaubt die Verwendung von MarkerClustern und Layern. Hier kann nach Belieben nur bestimmte Cluster getoggled werden und beim Rauszoomen werden nahe Markierungen zusammengefasst.

In [52]:
map = folium.Map(location=[df_ger_strong['latitude'].mean(), df_ger_strong['longitude'].mean()], zoom_start=4)

# Funktion zum Hinzufügen von Markern und Popup-Informationen
def add_markers(df, color, cluster):
    for index, row in df.iterrows():
        icon = folium.Icon(color=color, icon='info-sign')
        folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(cluster)

# Erstelle Marker-Cluster
cluster_very_strong = plugins.MarkerCluster(name='Very Strong Markers').add_to(map)
cluster_strong = plugins.MarkerCluster(name='Strong Markers').add_to(map)
cluster_weak = plugins.MarkerCluster(name='Weak Markers', color='orange').add_to(map)
cluster_very_weak = plugins.MarkerCluster(name='Very Weak Markers').add_to(map)

# Füge Marker zu den Clustern hinzu
add_markers(df_ger_very_strong, 'green', cluster_very_strong)
add_markers(df_ger_strong, 'lightgreen', cluster_strong)
add_markers(df_ger_weak, 'orange', cluster_weak)
add_markers(df_ger_very_weak, 'red', cluster_very_weak)

# Füge Marker-Cluster zur Karte hinzu
cluster_very_strong.add_to(map)
cluster_strong.add_to(map)
cluster_very_weak.add_to(map)
cluster_weak.add_to(map)

# Füge Layer Control zur Karte hinzu, um zwischen den Clustern zu wechseln
folium.LayerControl().add_to(map)
map

Für die Fragestellung lässt sich sagen, dass Deutschland in Turnieren innerhalb Europa und Amerika sehr große Schwankungen in ihrer Siegrate aufweist. Deutschland ist hier entweder besonders gut oder besonders schlecht je nach Turnier. Verblüffender Weise ist Deutschland in Amerika öfters stärker als schwächer, in Europa hingegen öfters schwächer als stärker. Für Asien (Japan) und Australien ist Deutschland jeweils sehr schwach bzw. schwach.